# Deploy Llama2 7b SmoothQuant Model with high performance on SageMaker using Sagemaker LMI



In this lab we explore how to host a Llama2 7b large language model with FP16 precision SmoothQuant quantized, on SageMaker using DeepSpeed. We use DJLServing as the model serving solution in this example that is bundled in the LMI container. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent blog post (https://aws.amazon.com/blogs/machine-learning/deploy-bloom-176b-and-opt-30b-on-amazon-sagemaker-with-large-model-inference-deep-learning-containers-and-deepspeed/).


Model parallelism can help deploy large models that would normally be too large for a single GPU. With model parallelism, we partition and distribute a model across multiple GPUs. Each GPU holds a different part of the model, resolving the memory capacity issue for the largest deep learning models with billions of parameters. 

SageMaker has rolled out DeepSpeed container which now provides users with the ability to leverage the managed serving capabilities and help to provide the un-differentiated heavy lifting.

In this notebook, we deploy https://huggingface.co/TheBloke/Llama-2-7b-fp16 model on a ml.g5.2xlarge instance. 

# Licence agreement
 - View license information https://huggingface.co/meta-llama before using the model.
 - This notebook is a sample notebook and not intended for production use. Please refer to the licence at https://github.com/aws/mit-0. 

In [2]:
%pip install -Uq sagemaker
%pip install -Uq transformers


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
model_bucket = sess.default_bucket()  # bucket to house model artifacts

llama2_7b_fp16_smoothquant_s3_code_prefix = "hf-large-model-djl/meta-llama/Llama2-7b-fp16-smoothquant/code"  # folder within bucket where code artifact will go

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

## Create SageMaker compatible Model artifact,  upload Model to S3 and bring your own inference script.

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or postprocessing of the model's predictions.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - serving.properties.

The tarball is in the following format:

```
code
├──── 
│   └── serving.properties
```

    serving.properties is the configuration file that can be used to configure the model server.


#### Create serving.properties 
This is a configuration file to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration.

Here is a list of settings that we use in this configuration file -

    engine: The engine for DJL to use. In this case, we have set it to MPI.
    option.model_id: The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models) or S3 path to the model artefacts. 
    option.tensor_parallel_degree: Set to the number of GPU devices over which Accelerate needs to partition the model. This parameter also controls the no of workers per model which will be started up when DJL serving runs. As an example if we have a 4 GPU machine and we are creating 4 partitions then we will have 1 worker per model to serve the requests.
    option.dtype: The precision with which you want to run inference.
    option.quantize: quantization method (bitsandbytes, gptq, smoothquant).

For more details on the configuration options and an exhaustive list, you can refer the documentation - https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html.



In [6]:
!rm -rf code_llama2_7b_fp16_smoothquant
!mkdir -p code_llama2_7b_fp16_smoothquant

In [7]:
%%writefile code_llama2_7b_fp16_smoothquant/serving.properties
engine=DeepSpeed
option.model_id=TheBloke/Llama-2-7B-fp16
option.tensor_parallel_degree=1
option.dtype=fp16
option.quantize=smoothquant

Writing code_llama2_7b_fp16_smoothquant/serving.properties


In [8]:
!pygmentize code_llama2_7b_fp16_smoothquant/serving.properties | cat -n

     1	engine=DeepSpeed
     2	option.model_id=TheBloke/Llama-2-7B-fp16
     3	option.tensor_parallel_degree=1
     4	option.dtype=fp16
     5	option.quantize=smoothquant


**Image URI for the DJL container is being used here**

We use `image_uris.retrieve` function to generate an ECR image URI for the pre-built SageMaker LMI container image.

In [9]:
try:
    inference_image_uri = image_uris.retrieve(
        framework="djl-deepspeed", region=region, version="0.25.0"
    )
except ValueError:
    inference_image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.25.0-deepspeed0.11.0-cu118"
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.25.0-deepspeed0.11.0-cu118


**Create the Tarball and then upload to S3 location**

In [10]:
!rm model.tar.gz
!tar czvf model.tar.gz code_llama2_7b_fp16_smoothquant

rm: cannot remove 'model.tar.gz': No such file or directory
code_llama2_7b_fp16_smoothquant/
code_llama2_7b_fp16_smoothquant/serving.properties


In [11]:
llama2_7b_fp16_smoothquant_s3_code_artifact = sess.upload_data("model.tar.gz", bucket, llama2_7b_fp16_smoothquant_s3_code_prefix)
llama2_7b_fp16_smoothquant_s3_code_artifact

's3://sagemaker-us-west-2-521058400674/hf-large-model-djl/meta-llama/Llama2-7b-fp16-smoothquant/code/model.tar.gz'

In [12]:
!aws s3 ls {llama2_7b_fp16_smoothquant_s3_code_artifact}

2023-12-06 21:33:52        272 model.tar.gz


### To create the endpoint the steps are:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.g5.2xlarge 
    
    b) ContainerStartupHealthCheckTimeoutInSeconds is 3600 to ensure health check starts after the model is ready    
3. Create the end point using the endpoint config created    


#### Create the Model
Use the image URI for the DJL container and the s3 location to which the tarball was uploaded.

The container downloads the model into the `/tmp` space on the instance because SageMaker maps the `/tmp` to the Amazon Elastic Block Store (Amazon EBS) volume that is mounted when we specify the endpoint creation parameter VolumeSizeInGB. 
It leverages `s5cmd`(https://github.com/peak/s5cmd) which offers a very fast download speed and hence extremely useful when downloading large models.

For instances like p4dn, which come pre-built with the volume instance, we can continue to leverage the `/tmp` on the container. The size of this mount is large enough to hold the model.


In [13]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"aim351-llama2-7b-smoothquant")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": llama2_7b_fp16_smoothquant_s3_code_artifact,
        "Environment": {"MODEL_LOADING_TIMEOUT": "3600"},
    },
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131
Created Model: arn:aws:sagemaker:us-west-2:521058400674:model/aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131


In [14]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:521058400674:endpoint-config/aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-config',
 'ResponseMetadata': {'RequestId': 'fd403a7d-2c4f-40b6-9128-dc5784defad1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fd403a7d-2c4f-40b6-9128-dc5784defad1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '140',
   'date': 'Wed, 06 Dec 2023 21:33:54 GMT'},
  'RetryAttempts': 0}}

In [15]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:521058400674:endpoint/aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint


### Deploy Llama 2 7B model

This step can take ~ 10 min or longer so please be patient

In [16]:
%%time
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:521058400674:endpoint/aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint
Status: InService
CPU times: user 110 ms, sys: 23.2 ms, total: 133 ms
Wall time: 7min 1s


#### While you wait for the endpoint to be created, you can read more about:
- [Deep Learning containers for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)

#### Leverage the Boto3 to invoke the endpoint. 

This is a generative model so we pass in a Text as a prompt and Model will complete the sentence and return the results.

You can pass a prompt as input to the model. This done by setting inputs to a prompt. The model then returns a result for each prompt. The text generation can be configured using appropriate parameters.
These parameters need to be passed to the endpoint as a dictionary of kwargs. Refer this documentation - https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig for more details.

The below code sample illustrates the invocation of the endpoint using a text prompt and also sets some parameters

In [17]:
system_message = """\
You are a helpful, respectful and honest assistant. 
Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information."""

prompt = "What is AWS reinvent? Where and when does this happen every year?"

prompt_template = f"""[INST] <<SYS>>{system_message}<</SYS>>\n{prompt}[/INST]"""

print(f"Prompt template: {prompt_template}")

Prompt template: [INST] <<SYS>>You are a helpful, respectful and honest assistant. 
Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.<</SYS>>
What is AWS reinvent? Where and when does this happen every year?[/INST]


#### Inference Results

Lets, invoke the model with the prompt defined above, body of the `invoke_endpoint()` parameter contains the prompt and the inference configurations including the following parameters

- `max_new_tokens` The maximum numbers of tokens to generate, ignoring the number of tokens in the prompt.
- `temperature`  The value used to modulate the next token probabilities.
- `return_full_text` if True this will return our original prompt along with the generated text

In [18]:
%%time

body = json.dumps(
        {
            "inputs": prompt_template,
            "parameters": {
                "max_new_tokens": 256,
                "temperature": 0.1
            },
        }
    )

response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=body,
    ContentType="application/json",
)

output = json.loads(response["Body"].read().decode("utf-8"))
output_response = output[0]["generated_text"]

print(f"Model Output: {output_response}")

Model Output: [INST] <<SYS>>You are a helpful, respectful and honest assistant. 
Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.<</SYS>>
What is AWS reinvent? Where and when does this happen every year?[/INST]

[INST] <<SYS>>AWS re:Invent is an annual conference hosted by Amazon Web Services (AWS) in Las Vegas, Nevada. 
The conference is held in late November and early December, and is attended by over 60,000 people. 
The conference features keynote speeches, breakout sessions, and networking opportunities for attendees. 
AWS re:Invent is a four-day event that includes keynote speec

Great, you have successfully run inference on the prompt and got the output. Note that, first invocation is going to be slower beacuse the model has to be loaded to GPU memory. Subsequent invocations should be faster to get a response from Llama 2 7B Chat quantized model with SmoothQuant when it is already loaded.

### Calculate number of output tokens

We will use AutoTokenizer from `transformers` package to load the tokenizer and calculate the number of token, we will use similar mechanism to benchmark the execution later in this notebook.

In [19]:
from transformers import AutoTokenizer
tokenizer_bnb = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7B-fp16")
print(f"Number of input tokens: {len(tokenizer_bnb.encode(prompt_template))}")
print(f"Number of output tokens: {len(tokenizer_bnb.encode(output_response))}")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

Number of input tokens: 157
Number of output tokens: 413


### Benchmarking Llama 2 7B  SmoothQuant quantized model

This following code is used to benchmark the speed of text generation for a given endpoint and tokenizer. We call the inference endpoint in a loop and keep track of the execution time and number of tokens generated to calculate tokens generated per second.


In [20]:
from tqdm.auto import tqdm
import time

def benchmark_generation_speed(endpoint_name, tokenizer):
    generation_time_list = []
    num_generated_tokens_list = []

    for i in tqdm(range(10)):
        print(f"Calling {endpoint_name} endpoint - #{i+1} time")
        start = time.time()
        resp = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=body,
            ContentType="application/json",
        )
        end = time.time()
        generation_time_list.append(end - start)
        output = json.loads(resp["Body"].read().decode("utf-8"))
        output_resp = output[0]["generated_text"]
        output_tokens = len(tokenizer.encode(output_resp))
        num_generated_tokens_list.append(output_tokens)

    total_tokens = sum(num_generated_tokens_list)
    total_seconds = sum(generation_time_list)
    total_tokens_per_second = total_tokens / total_seconds
    print(f"Generated {total_tokens} tokens using {total_seconds} seconds")
    print(f"Generation speed: {total_tokens_per_second} tokens/s") 
    
    return total_tokens, total_seconds, total_tokens_per_second

In [21]:
total_tokens, total_seconds, total_tokens_per_second = benchmark_generation_speed(endpoint_name, tokenizer_bnb)

  0%|          | 0/10 [00:00<?, ?it/s]

Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #1 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #2 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #3 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #4 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #5 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #6 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #7 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #8 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #9 time
Calling aim351-llama2-7b-smoothquant-2023-12-06-21-33-53-131-endpoint endpoint - #10 time
Generated 4130 tokens using 50.848403453826904 seconds
Generation speed: 81.22182250520929 tokens/s


We can see that for Llama2 7B FP16 quantized model with SmoothQuant is able to proces ~80 tokens per second. You will be able to optimize for latency using quantization technique.

### Congrats, you have suucessfully completed Lab2.

Please proceed to clean up section.

## Clean Up

Run the code below to delete the endpoint and avoid any additional charges.

In [22]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '67c5d582-6100-4060-970b-61a80c6ab5da',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '67c5d582-6100-4060-970b-61a80c6ab5da',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 06 Dec 2023 21:41:55 GMT'},
  'RetryAttempts': 0}}

In [23]:
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '8786785d-98a4-461f-9bf0-853ec166c670',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8786785d-98a4-461f-9bf0-853ec166c670',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 06 Dec 2023 21:41:55 GMT'},
  'RetryAttempts': 0}}